# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.metrics.distance import edit_distance
import random
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [2]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

In [3]:
with open('litw-win.txt', encoding = 'cp1251') as f:
    words1 = [row.split()[-1] for row in f]
text_words = list(text.split())
for i in range(len(text_words)):
    if text_words[i] not in words1:
        distances = {j : edit_distance(text_words[i], j) for j in words1}
        sorted_distances = sorted(distances.items(), key = lambda x: x[1])
        text_words[i] = sorted_distances[0][0]
print(' '.join(text_words))

с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий


2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [4]:
text2 = 'Считайте слова из файла litw-win.txt и запишите их в список words. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка words. Считайте, что в слове есть опечатка, если данное слово не содержится в списке words'

In [5]:
stemmer = SnowballStemmer('russian')
morph = pymorphy2.MorphAnalyzer()
lemmatizer = WordNetLemmatizer()
text2_words = [word for word in word_tokenize(str(text2)) if word.isalpha()]

In [6]:
for i in text2_words:
    print(stemmer.stem(i))

счита
слов
из
файл
и
запиш
их
в
список
words
в
зада
предложен
исправьт
все
опечатк
замен
слов
с
опечатк
на
ближайш
в
смысл
расстоян
левенштейн
к
ним
слов
из
списк
words
счита
что
в
слов
ест
опечатк
есл
дан
слов
не
содерж
в
списк
words


In [7]:
for i in text2_words:
    print(morph.parse(i)[0].normalized.word)

считать
слово
из
файл
и
записать
они
в
список
words
в
задать
предложение
исправить
всё
опечатка
заменить
слово
с
опечатка
на
близкий
в
смысл
расстояние
левенштейн
к
они
слово
из
список
words
считать
что
в
слово
есть
опечатка
если
данный
слово
не
содержаться
в
список
words


3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [9]:
import nltk
cv = CountVectorizer()
cv.fit_transform(nltk.sent_tokenize(text2)).toarray()

array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]], dtype=int64)

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [10]:
data = pd.read_csv('preprocessed_descriptions.csv')
data

,Unnamed: 0,name,preprocessed_descriptions
0,0,george s at the cove black bean soup,an original recipe created by chef scott meska...
1,1,healthy for them yogurt popsicles,my children and their friends ask for my homem...
2,2,i can t believe it s spinach,these were so go it surprised even me
3,3,italian gut busters,my sisterinlaw made these for us at a family g...
4,4,love is in the air beef fondue sauces,i think a fondue is a very romantic casual din...
...,...,...,...
29995,29995,zurie s holey rustic olive and cheddar bread,this is based on a french recipe but i changed...
29996,29996,zwetschgenkuchen bavarian plum cake,this is a traditional fresh plum cake thought ...
29997,29997,zwiebelkuchen southwest german onion cake,this is a traditional late summer early fall s...
29998,29998,zydeco soup,this is a delicious soup that i originally fou...


In [11]:
all_words = []
for i in data['preprocessed_descriptions']:
    all_words.extend([word for word in word_tokenize(str(i)) if word.isalpha()])
words = set(all_words)
print(words)

{'herbfarm', 'crostiniwith', 'meatsalads', 'ristorante', 'a', 'boggled', 'deluxe', 'disapointed', 'rheinlander', 'expense', 'mad', 'unusually', 'crocks', 'abroad', 'steen', 'life', 'stwerart', 'crimes', 'valley', 'veggiefruit', 'cocounut', 'itramen', 'spin', 'shadings', 'mmmmlemon', 'kaf', 'differently', 'shareserve', 'superfoodthe', 'soyfree', 'wimbledonstrawberriescom', 'abbey', 'guessing', 'vessel', 'usally', 'goodsized', 'complicated', 'peasanttype', 'guacamolesee', 'postgym', 'frangelico', 'marascara', 'martini', 'feeling', 'sci', 'lawn', 'optimal', 'stimulating', 'accomplishment', 'freezingrefrigerating', 'sop', 'burlapidahos', 'marjolaine', 'emperor', 'torgo', 'drinkswap', 'atkins', 'pearls', 'thorough', 'cheyenne', 'donated', 'powdery', 'neglect', 'rawcooking', 'pham', 'significant', 'nuance', 'pius', 'fair', 'foradditional', 'flown', 'wheatfreeglutenfree', 'author', 'monticello', 'reef', 'brothserves', 'cookbooksoriginally', 'gruszecki', 'soviet', 'downand', 'korean', 'discuss

In [68]:
len(words)

30743

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [12]:
for i in range(5):
    w1, w2 = random.sample(list(words), 2)
    print(f'Слова: {w1}, {w2}. Расстояние редактирования: {edit_distance(w1, w2)}\n')

Слова: entryin, dhansak. Расстояние редактирования: 7

Слова: amen, waynesville. Расстояние редактирования: 9

Слова: salsait, purity. Расстояние редактирования: 6

Слова: countless, pairs. Расстояние редактирования: 8

Слова: frumceliac, conditions. Расстояние редактирования: 10



1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [26]:
def dist(word, k):
    dists = {i : edit_distance(word, i) for i in words}
    sorted_dists = sorted(dists.items(), key = lambda x: x[1])
    return sorted_dists[:k]

In [27]:
dist('database', 5)

[('database', 0),
 ('patatas', 3),
 ('oatbased', 3),
 ('details', 4),
 ('garbage', 4)]

In [28]:
dist('description', 7)

[('description', 0),
 ('descriptions', 1),
 ('desciption', 1),
 ('prescription', 2),
 ('descriptive', 2),
 ('destruction', 3),
 ('discretion', 3)]

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [13]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()
df_words = pd.DataFrame(words, columns = ['word'])
df_words['stemmed_word'] = df_words.apply(lambda x: stemmer.stem(x['word']), axis = 1)
df_words['normalized_word'] = df_words.apply(lambda x: lemmatizer.lemmatize(x['word']), axis = 1)
df_words = df_words.set_index('word')
df_words

,stemmed_word,normalized_word
word,,
herbfarm,herbfarm,herbfarm
crostiniwith,crostiniwith,crostiniwith
meatsalads,meatsalad,meatsalads
ristorante,ristorant,ristorante
a,a,a
...,...,...
foam,foam,foam
collectionshe,collectionsh,collectionshe
treasure,treasur,treasure


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [14]:
stop_words = stopwords.words('english')
words_no_stop = [word for word in all_words if word not in stop_words]
print(f'Доля стоп слов составляла: {(len(all_words) - len(words_no_stop)) / len(all_words) * 100}')

Доля стоп слов составляла: 46.10722870409009


In [80]:
for k, v in nltk.FreqDist(all_words).most_common(10):
    print(f'{k} - {v}')

the - 40072
a - 34951
and - 30245
this - 26859
i - 24836
to - 23471
is - 20285
it - 19756
of - 18364
for - 15939


In [81]:
for k, v in nltk.FreqDist(words_no_stop).most_common(10):
    print(f'{k} - {v}')

recipe - 14871
make - 6326
time - 5137
use - 4620
great - 4430
like - 4167
easy - 4152
one - 3872
made - 3810
good - 3791


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [15]:
data3 = data.sample(5)
data3

,Unnamed: 0,name,preprocessed_descriptions
10861,10861,favorite roti meat filling,this is my husbands favorite meat filling for ...
14137,14137,hot and sour tofu soup,NaN
21118,21118,poohrona s squash n onions,light and full of flavor this dish is one of m...
23811,23811,seed spelt bread abm,originally from 300 best canadian bread machin...
11012,11012,fiore with broccoli rabe chicken and pecorin...,this light and wellseasoned recipe contains mo...


In [106]:
tv = TfidfVectorizer()
for i in data3['preprocessed_descriptions']:
    print(tv.fit_transform(sent_tokenize(i)).toarray(), '\n')

[[0.13483997 0.26967994 0.40451992 0.13483997 0.13483997 0.13483997
  0.13483997 0.13483997 0.13483997 0.13483997 0.13483997 0.13483997
  0.13483997 0.13483997 0.13483997 0.13483997 0.13483997 0.13483997
  0.13483997 0.26967994 0.26967994 0.13483997 0.13483997 0.13483997
  0.13483997 0.13483997 0.13483997 0.13483997 0.13483997 0.13483997
  0.13483997 0.13483997 0.13483997 0.13483997 0.13483997 0.13483997
  0.13483997 0.13483997]] 

[[0.11111111 0.11111111 0.11111111 0.11111111 0.22222222 0.11111111
  0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
  0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.22222222
  0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
  0.11111111 0.11111111 0.11111111 0.22222222 0.11111111 0.11111111
  0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
  0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
  0.11111111 0.11111111 0.11111111 0.22222222 0.11111111 0.11111111
  0.22222222 0.11111

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [16]:
from scipy.spatial.distance import cosine
data3['preprocessed_descriptions'] = data3['preprocessed_descriptions'].astype(str)
tv = TfidfVectorizer()
vectors = tv.fit_transform(data3['preprocessed_descriptions']).toarray()
df3_2 = pd.DataFrame(index = data3['name'], columns = data3['name'])
for i, r1 in enumerate(data3['name']):
    for j, r2 in enumerate(data3['name']):
        df3_2.at[r1, r2] = 1 - cosine(vectors[i], vectors[j])
df3_2

name,favorite roti meat filling,hot and sour tofu soup,poohrona s squash n onions,seed spelt bread abm,fiore with broccoli rabe chicken and pecorino cheese
name,,,,,
favorite roti meat filling,1,0.0,0.218255,0.112809,0.087074
hot and sour tofu soup,0.0,1,0.0,0.0,0.0
poohrona s squash n onions,0.218255,0.0,1,0.074078,0.156697
seed spelt bread abm,0.112809,0.0,0.074078,1,0.082922
fiore with broccoli rabe chicken and pecorino cheese,0.087074,0.0,0.156697,0.082922,1


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [17]:
res = df3_2.where(df3_2 != 1).max().max()
res

0.21825491593970925